In [26]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import pearsonr, spearmanr
from scipy import stats
from IPython.display import display, HTML
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, classification_report
from sklearn.model_selection import train_test_split
from datetime import datetime
from statsmodels.formula.api import ols
from statsmodels.stats.api import anova_lm
from IPython.display import display, HTML
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 3)
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>div.output_scroll { height: unset; }</style>"))
display(HTML("<style>pre { white-space: pre !important; }</style>"))
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer




In [27]:
#loading the dataset
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None)

df_trustpilot = pd.read_csv('data/data_trustpilot_2.csv', engine='python')

display(df_trustpilot.head())



,rating,location,username,number_reviews,verification,repeat_reviewer,repeat_reviewer_encoded,company,text,text_processed,text_length,text_word_length,Sentiment_Blob,Sentiment_Blob_cat,Sentiment_VADER,Sentiment_VADER_cat,subject,subject_processed,subject_length,subject_word_length,answer,answer_processed,answered_encoded,date_of_experience,date_posted,local_date_posted,month_local,local_hour,time_of_day,day_of_week_posted,day_type,days_between_experience_and_post,review_time,review_time_encoded
0,5,CA,Rob Crane,2,Redirected,repeat,1,Flashbay,The company rep I worked with made my transaction smooth and quick. The samples were so helpful and the item shipping was relatively very fast. I will be a repeat customer.,company rep worked made transaction smooth quick samples helpful item shipping relatively fast repeat customer,110,15,0.311,4.0,0.421,4,The company rep I worked with made my…,company rep worked made,23,4,0,0,0,2024-06-15,2024-10-23 04:17:44,2024-10-22,10,21,Evening,1,Business Day,129,late_review,0
1,5,US,Pat Anderson,1,Verified,one-time,0,Flashbay,I highly recommend using Flashbay. Immediately after ordering I heard from them and they were attentive to every little detail. I had notebooks shipped to Alaska for a conference and they arrived in perfect condition and delivered on time! Can't say enough good things about Flashbay!,highly recommend using flashbay immediately ordering heard attentive every little detail notebooks shipped alaska conference arrived perfect condition delivered time cant say enough good things flashbay,202,26,0.345,4.0,0.623,5,I highly recommend using Flashbay,highly recommend using flashbay,31,4,0,0,0,2024-10-16,2024-10-16 19:34:05,2024-10-16,10,12,Business Hours,2,Business Day,0,quick_review,1
2,5,CZ,Margarita Orlova,1,Verified,one-time,0,Flashbay,"I had the pleasure of working with Shelby Gibson on a large order that we needed to ship to different locations. Shelby was very responsive, very kind and helpful. The order process was smooth and the final products look great. Only positive feedback from my side.",pleasure working shelby gibson large order needed ship different locations shelby responsive kind helpful order process smooth final products look great positive feedback side,175,24,0.320,4.0,0.964,5,Great customer service,great customer service,22,3,0,0,0,2024-10-10,2024-10-17 10:27:44,2024-10-17,10,10,Business Hours,3,Business Day,7,late_review,0
3,5,US,Paola Rivas,1,Verified,one-time,0,Flashbay,"I had a fantastic experience with Brian Truong. He was attentive, truly understood my needs, and provided all the support I needed. Excellent customer service—thank you!",fantastic experience brian truong attentive truly understood needs provided support needed excellent customer servicethank,122,14,0.600,4.0,0.917,5,Outstanding Support and Attentive Service,outstanding support attentive service,37,4,0,0,0,2024-10-21,2024-10-21 22:38:50,2024-10-21,10,15,Business Hours,0,Business Day,0,quick_review,1
4,5,CA,Fiona Mckelvey Keenan,3,Not Verified,repeat,1,Flashbay,"My number-one go-to for computer accessories. Rachel was superb. She responded to my every request immediately, and her communication was satisfactory enough to get all the answers I wanted. Highly recommend!",numberone goto computer accessories rachel superb responded every request immediately communication satisfactory enough get answers wanted highly recommend,155,18,0.387,4.0,0.855,5,My number-one go-to for computer…,numberone goto computer,23,3,0,0,0,2024-07-11,2024-10-23 04:09:05,2024-10-22,10,21,Evening,1,Business Day,103,late_review,0


In [28]:
#First check
df_trustpilot['local_date_posted'] = pd.to_datetime(df_trustpilot['local_date_posted'])
df_trustpilot['date_of_experience'] = pd.to_datetime(df_trustpilot['date_of_experience'])
df_trustpilot['date_posted'] = pd.to_datetime(df_trustpilot['date_posted'])

print(df_trustpilot.columns)
display(df_trustpilot.info())

Index(['rating', 'location', 'username', 'number_reviews', 'verification',
       'repeat_reviewer', 'repeat_reviewer_encoded', 'company', 'text',
       'text_processed', 'text_length', 'text_word_length', 'Sentiment_Blob',
       'Sentiment_Blob_cat', 'Sentiment_VADER', 'Sentiment_VADER_cat',
       'subject', 'subject_processed', 'subject_length', 'subject_word_length',
       'answer', 'answer_processed', 'answered_encoded', 'date_of_experience',
       'date_posted', 'local_date_posted', 'month_local', 'local_hour',
       'time_of_day', 'day_of_week_posted', 'day_type',
       'days_between_experience_and_post', 'review_time',
       'review_time_encoded'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64414 entries, 0 to 64413
Data columns (total 34 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   rating                            64414 non-null  in

None

STAGE 1 - TRAIN/TEST SPLIT

In [29]:
# Splitting data with stratification on rating to make sure that ratings are distributed evenly across both sets due to data imbalance
train_df, test_df = train_test_split(df_trustpilot, test_size=0.2,  
    random_state=42, stratify=df_trustpilot['rating'])

print("Training set size:", len(train_df))
print("Test set size:", len(test_df))

# Verifying rating distribution
print("Rating distribution in original data:")
print(df_trustpilot['rating'].value_counts(normalize=True).round(3))
print("Rating distribution in training set:")
print(train_df['rating'].value_counts(normalize=True).round(3))
print("Rating distribution in test set:")
print(test_df['rating'].value_counts(normalize=True).round(3))

Training set size: 51531
Test set size: 12883
Rating distribution in original data:
rating
5    0.487
1    0.251
4    0.098
3    0.097
2    0.067
Name: proportion, dtype: float64
Rating distribution in training set:
rating
5    0.487
1    0.251
4    0.098
3    0.097
2    0.067
Name: proportion, dtype: float64
Rating distribution in test set:
rating
5    0.487
1    0.251
4    0.098
3    0.097
2    0.067
Name: proportion, dtype: float64


STAGE 2 - HANDLE OUTLIERS

In [31]:
# Handle outliers on numeric columns that make sense
numeric_cols = ['number_reviews', 'days_between_experience_and_post',
                'text_length', 'text_word_length', 'subject_length', 
                'subject_word_length']

for col in numeric_cols:
    # Calculate bounds using training data only
    Q1 = train_df[col].quantile(0.25)
    Q3 = train_df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Print outlier info for training data
    outliers = train_df[(train_df[col] < lower_bound) | (train_df[col] > upper_bound)]
    print(f"\n{col} training outliers: {len(outliers)} ({(len(outliers)/len(train_df)*100):.2f}%)")
    
    # Clip the values in place so they become the upper or lower bound
    train_df[col] = train_df[col].clip(lower_bound, upper_bound)
    test_df[col] = test_df[col].clip(lower_bound, upper_bound)


for col in numeric_cols:
    Q1 = train_df[col].quantile(0.25)
    Q3 = train_df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = train_df[(train_df[col] < lower_bound) | (train_df[col] > upper_bound)]
    if len(outliers) > 0:
        print(f"Warning: {col} still has {len(outliers)} outliers")
    else:
        print(f"{col}: No outliers remain")



number_reviews outliers: 6604 (10.25%)

days_between_experience_and_post outliers: 17625 (27.36%)

text_length outliers: 4588 (7.12%)

text_word_length outliers: 4502 (6.99%)

subject_length outliers: 2541 (3.94%)

subject_word_length outliers: 6653 (10.33%)
number_reviews: No outliers remain
days_between_experience_and_post: No outliers remain
text_length: No outliers remain
text_word_length: No outliers remain
subject_length: No outliers remain
subject_word_length: No outliers remain


STAGE 3 - ADVANCED TEXT PREPROCESSING

In [ ]:
# TF-IDF with unigrams and bigrams
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2),
    max_df=0.95, min_df=2)

# Fit and transform training data
X_train_tfidf = tfidf.fit_transform(train_df['text_processed'])
X_test_tfidf = tfidf.transform(test_df['text_processed'])

np.save('data/X_train_tfidf.npy', X_train_tfidf.toarray())
np.save('data/X_test_tfidf.npy', X_test_tfidf.toarray())
np.save('data/tfidf_features.npy', tfidf.get_feature_names_out())

# Get feature names (top n-grams)
feature_names = tfidf.get_feature_names_out()
print(f"Number of features: {len(feature_names)}")

# Optional: Convert to DataFrame for visualization
tfidf_df = pd.DataFrame(X_train_tfidf.toarray(), columns=feature_names)
print("Top 10 n-grams:")
print(tfidf_df.sum().sort_values(ascending=False).head(10))

# For TF-IDF output:
print("TF-IDF Feature Overview:")
print(f"Number of features (n-grams): {len(tfidf.get_feature_names_out())}")
print("Top 10 most important n-grams:")
feature_importance = pd.DataFrame({
    'feature': tfidf.get_feature_names_out(),
    'importance': X_train_tfidf.sum(axis=0).A1
}).sort_values('importance', ascending=False).head(10)
print(feature_importance)

In [32]:
#Inspecting final output 

print(df_trustpilot.columns)
print(df_trustpilot.info())
print("The Trustpilot dataset has ", df_trustpilot.shape[1], "columns")
print("The Trustpilot dataset has ", df_trustpilot.shape[0], "rows")
display(df_trustpilot.head())

Index(['rating', 'location', 'username', 'number_reviews', 'verification',
       'repeat_reviewer', 'repeat_reviewer_encoded', 'company', 'text',
       'text_processed', 'text_length', 'text_word_length', 'Sentiment_Blob',
       'Sentiment_Blob_cat', 'Sentiment_VADER', 'Sentiment_VADER_cat',
       'subject', 'subject_processed', 'subject_length', 'subject_word_length',
       'answer', 'answer_processed', 'answered_encoded', 'date_of_experience',
       'date_posted', 'local_date_posted', 'month_local', 'local_hour',
       'time_of_day', 'day_of_week_posted', 'day_type',
       'days_between_experience_and_post', 'review_time',
       'review_time_encoded'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64414 entries, 0 to 64413
Data columns (total 34 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   rating                            64414 non-null  in

,rating,location,username,number_reviews,verification,repeat_reviewer,repeat_reviewer_encoded,company,text,text_processed,text_length,text_word_length,Sentiment_Blob,Sentiment_Blob_cat,Sentiment_VADER,Sentiment_VADER_cat,subject,subject_processed,subject_length,subject_word_length,answer,answer_processed,answered_encoded,date_of_experience,date_posted,local_date_posted,month_local,local_hour,time_of_day,day_of_week_posted,day_type,days_between_experience_and_post,review_time,review_time_encoded
0,5,CA,Rob Crane,2,Redirected,repeat,1,Flashbay,The company rep I worked with made my transaction smooth and quick. The samples were so helpful and the item shipping was relatively very fast. I will be a repeat customer.,company rep worked made transaction smooth quick samples helpful item shipping relatively fast repeat customer,110,15,0.311,4.0,0.421,4,The company rep I worked with made my…,company rep worked made,23.0,4.0,0,0,0,2024-06-15,2024-10-23 04:17:44,2024-10-22,10,21,Evening,1,Business Day,0,late_review,0
1,5,US,Pat Anderson,1,Verified,one-time,0,Flashbay,I highly recommend using Flashbay. Immediately after ordering I heard from them and they were attentive to every little detail. I had notebooks shipped to Alaska for a conference and they arrived in perfect condition and delivered on time! Can't say enough good things about Flashbay!,highly recommend using flashbay immediately ordering heard attentive every little detail notebooks shipped alaska conference arrived perfect condition delivered time cant say enough good things flashbay,202,26,0.345,4.0,0.623,5,I highly recommend using Flashbay,highly recommend using flashbay,31.0,4.0,0,0,0,2024-10-16,2024-10-16 19:34:05,2024-10-16,10,12,Business Hours,2,Business Day,0,quick_review,1
2,5,CZ,Margarita Orlova,1,Verified,one-time,0,Flashbay,"I had the pleasure of working with Shelby Gibson on a large order that we needed to ship to different locations. Shelby was very responsive, very kind and helpful. The order process was smooth and the final products look great. Only positive feedback from my side.",pleasure working shelby gibson large order needed ship different locations shelby responsive kind helpful order process smooth final products look great positive feedback side,175,24,0.320,4.0,0.964,5,Great customer service,great customer service,22.0,3.0,0,0,0,2024-10-10,2024-10-17 10:27:44,2024-10-17,10,10,Business Hours,3,Business Day,0,late_review,0
3,5,US,Paola Rivas,1,Verified,one-time,0,Flashbay,"I had a fantastic experience with Brian Truong. He was attentive, truly understood my needs, and provided all the support I needed. Excellent customer service—thank you!",fantastic experience brian truong attentive truly understood needs provided support needed excellent customer servicethank,122,14,0.600,4.0,0.917,5,Outstanding Support and Attentive Service,outstanding support attentive service,37.0,4.0,0,0,0,2024-10-21,2024-10-21 22:38:50,2024-10-21,10,15,Business Hours,0,Business Day,0,quick_review,1
4,5,CA,Fiona Mckelvey Keenan,3,Not Verified,repeat,1,Flashbay,"My number-one go-to for computer accessories. Rachel was superb. She responded to my every request immediately, and her communication was satisfactory enough to get all the answers I wanted. Highly recommend!",numberone goto computer accessories rachel superb responded every request immediately communication satisfactory enough get answers wanted highly recommend,155,18,0.387,4.0,0.855,5,My number-one go-to for computer…,numberone goto computer,23.0,3.0,0,0,0,2024-07-11,2024-10-23 04:09:05,2024-10-22,10,21,Evening,1,Business Day,0,late_review,0


In [33]:
import os
os.makedirs('data', exist_ok=True)

csv_path = 'data/data_trustpilot_3.csv'
df_trustpilot.to_csv(csv_path, index=False)
print(f"Full DataFrame exported to {csv_path}")

train_path = 'data/train_trustpilot_3.csv'
test_path = 'data/test_trustpilot_3.csv'

train_df.to_csv(train_path, index=False)
test_df.to_csv(test_path, index=False)

print(f"Training set exported to {train_path}")
print(f"Test set exported to {test_path}")

X_train_tfidf = np.load('data/X_train_tfidf.npy')
X_test_tfidf = np.load('data/X_test_tfidf.npy')
feature_names = np.load('data/tfidf_features.npy', allow_pickle=True)


Full DataFrame exported to data/data_trustpilot_3.csv
Training set exported to data/train_trustpilot_3.csv
Test set exported to data/test_trustpilot_3.csv
